# Data Loading Basics for AG News Text Classification

## Overview

This notebook demonstrates fundamental data loading techniques following methodologies from:
- Zhang et al. (2015): "Character-level Convolutional Networks for Text Classification"
- Joulin et al. (2017): "Bag of Tricks for Efficient Text Classification"
- Wolf et al. (2020): "Transformers: State-of-the-Art Natural Language Processing"

### Tutorial Objectives
1. Load AG News dataset from multiple sources
2. Understand data structure and format
3. Create efficient data pipelines
4. Implement data validation
5. Optimize loading performance
6. Save processed data for training

Author: Võ Hải Dũng  
Email: vohaidung.work@gmail.com  
Date: 2025

## 1. Environment Setup

In [ ]:
# Standard library imports
import sys
import os
import time
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
import warnings

# Data manipulation imports
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

# Project imports
PROJECT_ROOT = Path("../..").resolve()
sys.path.insert(0, str(PROJECT_ROOT))

from src.data.datasets.ag_news import AGNewsDataset, AGNewsConfig, create_ag_news_datasets
from src.data.datasets.external_news import ExternalNewsDataset
from src.data.datasets.combined_dataset import CombinedDataset
from src.data.loaders.dataloader import create_dataloaders, DataLoaderConfig
from src.data.loaders.dynamic_batching import DynamicBatchSampler
from src.data.loaders.prefetch_loader import PrefetchLoader
from src.utils.io_utils import safe_load, safe_save, ensure_dir
from src.utils.logging_config import setup_logging
from src.utils.reproducibility import set_seed
from configs.config_loader import ConfigLoader
from configs.constants import (
    AG_NEWS_CLASSES,
    AG_NEWS_NUM_CLASSES,
    LABEL_TO_ID,
    ID_TO_LABEL,
    DATA_DIR
)

# Configuration
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
set_seed(42)
logger = setup_logging('data_loading_tutorial')

print("Data Loading Tutorial")
print("="*50)
print(f"Project Root: {PROJECT_ROOT}")
print(f"Data Directory: {DATA_DIR}")

## 2. Load Configuration

In [ ]:
# Load data loading configuration
config_loader = ConfigLoader()

# Load data config
data_config = config_loader.load_config('data/preprocessing/standard.yaml')

# Tutorial configuration
tutorial_config = {
    'max_samples': 1000,
    'batch_size': 16,
    'num_workers': 2,
    'pin_memory': torch.cuda.is_available(),
    'use_cache': True,
    'validate_data': True
}

print("Data Loading Configuration:")
print("="*50)
for key, value in tutorial_config.items():
    print(f"  {key}: {value}")

## 3. Loading from Processed Files

In [ ]:
# Method 1: Load from processed files
def load_from_processed() -> Tuple[AGNewsDataset, AGNewsDataset, AGNewsDataset]:
    """
    Load AG News dataset from preprocessed files.
    
    Following data loading best practices from:
        TensorFlow Data Validation (TFDV) documentation
    """
    config = AGNewsConfig(
        data_dir=DATA_DIR / "processed",
        max_samples=tutorial_config['max_samples'],
        validate_labels=tutorial_config['validate_data'],
        use_cache=tutorial_config['use_cache']
    )
    
    train_dataset = AGNewsDataset(config, split="train")
    val_dataset = AGNewsDataset(config, split="validation")
    test_dataset = AGNewsDataset(config, split="test")
    
    return train_dataset, val_dataset, test_dataset

print("Loading from processed files...")
start_time = time.time()

try:
    train_dataset, val_dataset, test_dataset = load_from_processed()
    load_time = time.time() - start_time
    
    print(f"Loading completed in {load_time:.2f} seconds")
    print(f"\nDataset sizes:")
    print(f"  Train: {len(train_dataset):,} samples")
    print(f"  Validation: {len(val_dataset):,} samples")
    print(f"  Test: {len(test_dataset):,} samples")
    
except Exception as e:
    print(f"Error loading processed data: {e}")
    print("Please run data preparation scripts first.")

## 4. Loading from Hugging Face Hub

In [ ]:
# Method 2: Load from Hugging Face datasets
def load_from_huggingface() -> Dict[str, Any]:
    """
    Load AG News dataset from Hugging Face Hub.
    
    References:
        Lhoest et al. (2021): "Datasets: A Community Library for Natural Language Processing"
    """
    from datasets import load_dataset
    
    print("Downloading from Hugging Face Hub...")
    dataset = load_dataset('ag_news')
    
    return dataset

# Try loading from Hugging Face
try:
    hf_dataset = load_from_huggingface()
    
    print("\nHugging Face dataset structure:")
    print(f"  Splits: {list(hf_dataset.keys())}")
    print(f"  Features: {hf_dataset['train'].features}")
    print(f"  Train size: {len(hf_dataset['train']):,}")
    print(f"  Test size: {len(hf_dataset['test']):,}")
    
    # Show sample
    sample = hf_dataset['train'][0]
    print(f"\nSample data:")
    print(f"  Label: {sample['label']} ({ID_TO_LABEL[sample['label']]})")
    print(f"  Text: {sample['text'][:200]}...")
    
except Exception as e:
    print(f"Could not load from Hugging Face: {e}")

## 5. Custom Dataset Implementation

In [ ]:
# Create custom dataset wrapper
class AGNewsCustomDataset(Dataset):
    """
    Custom PyTorch Dataset implementation for AG News.
    
    Following PyTorch dataset design patterns from:
        Paszke et al. (2019): "PyTorch: An Imperative Style, High-Performance Deep Learning Library"
    """
    
    def __init__(self, texts: List[str], labels: List[int], tokenizer=None, max_length: int = 256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        text = self.texts[idx]
        label = self.labels[idx]
        
        if self.tokenizer:
            encoding = self.tokenizer(
                text,
                truncation=True,
                padding='max_length',
                max_length=self.max_length,
                return_tensors='pt'
            )
            
            return {
                'input_ids': encoding['input_ids'].squeeze(),
                'attention_mask': encoding['attention_mask'].squeeze(),
                'labels': torch.tensor(label, dtype=torch.long)
            }
        else:
            return {
                'text': text,
                'labels': torch.tensor(label, dtype=torch.long)
            }

# Create sample custom dataset
sample_texts = train_dataset.texts[:100]
sample_labels = train_dataset.labels[:100]

custom_dataset = AGNewsCustomDataset(
    texts=sample_texts,
    labels=sample_labels
)

print("Custom Dataset Created:")
print(f"  Size: {len(custom_dataset)}")
print(f"  Sample item keys: {list(custom_dataset[0].keys())}")

## 6. DataLoader Configuration

In [ ]:
# Configure DataLoader
def create_optimized_dataloader(
    dataset: Dataset,
    batch_size: int = 32,
    shuffle: bool = True,
    num_workers: int = 4
) -> DataLoader:
    """
    Create optimized DataLoader with best practices.
    
    Following optimization strategies from:
        Li et al. (2020): "PyTorch Distributed: Experiences on Accelerating Data Parallel Training"
    """
    dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=torch.cuda.is_available(),
        drop_last=False,
        prefetch_factor=2 if num_workers > 0 else 2,
        persistent_workers=True if num_workers > 0 else False
    )
    
    return dataloader

# Create DataLoaders
train_loader = create_optimized_dataloader(
    custom_dataset,
    batch_size=tutorial_config['batch_size'],
    shuffle=True,
    num_workers=tutorial_config['num_workers']
)

print("DataLoader Configuration:")
print(f"  Batch size: {tutorial_config['batch_size']}")
print(f"  Number of batches: {len(train_loader)}")
print(f"  Pin memory: {torch.cuda.is_available()}")
print(f"  Number of workers: {tutorial_config['num_workers']}")

# Test DataLoader
print("\nTesting DataLoader...")
batch = next(iter(train_loader))
print(f"  Batch keys: {list(batch.keys())}")
print(f"  Batch labels shape: {batch['labels'].shape}")

## 7. Memory-Efficient Loading

In [ ]:
# Implement memory-efficient loading
class LazyAGNewsDataset(Dataset):
    """
    Lazy loading dataset to minimize memory usage.
    
    Following memory optimization techniques from:
        Chen et al. (2016): "Training Deep Nets with Sublinear Memory Cost"
    """
    
    def __init__(self, data_file: Path, max_samples: Optional[int] = None):
        self.data_file = data_file
        self.max_samples = max_samples
        
        # Only load metadata
        self._load_metadata()
        
    def _load_metadata(self):
        """Load only essential metadata."""
        # Simulate loading metadata
        self.length = self.max_samples if self.max_samples else 120000
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx: int) -> Dict[str, Any]:
        """Load data on-demand."""
        # Simulate loading single item from disk
        text = f"Sample news article {idx}"
        label = idx % AG_NEWS_NUM_CLASSES
        
        return {
            'text': text,
            'label': label
        }

# Create lazy dataset
lazy_dataset = LazyAGNewsDataset(
    data_file=DATA_DIR / "processed" / "train.json",
    max_samples=10000
)

print("Lazy Loading Dataset:")
print(f"  Dataset size: {len(lazy_dataset):,}")
print(f"  Memory efficient: Yes")
print(f"  On-demand loading: Yes")

# Memory usage comparison
import sys
regular_size = sys.getsizeof(train_dataset.texts) + sys.getsizeof(train_dataset.labels)
lazy_size = sys.getsizeof(lazy_dataset)

print(f"\nMemory Usage Comparison:")
print(f"  Regular dataset: ~{regular_size / 1024**2:.2f} MB")
print(f"  Lazy dataset: ~{lazy_size / 1024**2:.2f} MB")
print(f"  Reduction: {(1 - lazy_size/regular_size)*100:.1f}%")

## 8. Data Validation and Quality Checks

In [ ]:
def validate_dataset(dataset: Dataset) -> Dict[str, Any]:
    """
    Perform comprehensive data validation.
    
    Following validation practices from:
        Breck et al. (2019): "Data Validation for Machine Learning"
    """
    validation_results = {
        'passed': True,
        'issues': [],
        'statistics': {}
    }
    
    # Check dataset size
    if len(dataset) == 0:
        validation_results['passed'] = False
        validation_results['issues'].append("Dataset is empty")
    
    # Check label distribution
    labels = [dataset[i]['labels'].item() if torch.is_tensor(dataset[i]['labels']) 
              else dataset[i]['labels'] for i in range(min(1000, len(dataset)))]
    
    label_counts = pd.Series(labels).value_counts()
    validation_results['statistics']['label_distribution'] = label_counts.to_dict()
    
    # Check for invalid labels
    invalid_labels = [l for l in labels if l < 0 or l >= AG_NEWS_NUM_CLASSES]
    if invalid_labels:
        validation_results['passed'] = False
        validation_results['issues'].append(f"Found {len(invalid_labels)} invalid labels")
    
    # Check for missing data
    sample_issues = 0
    for i in range(min(100, len(dataset))):
        sample = dataset[i]
        if 'text' in sample and not sample['text']:
            sample_issues += 1
    
    if sample_issues > 0:
        validation_results['issues'].append(f"Found {sample_issues} samples with empty text")
    
    return validation_results

# Validate datasets
print("Dataset Validation Results:")
print("="*50)

for name, dataset in [("Custom", custom_dataset), ("Lazy", lazy_dataset)]:
    results = validate_dataset(dataset)
    
    print(f"\n{name} Dataset:")
    print(f"  Validation passed: {results['passed']}")
    
    if results['issues']:
        print(f"  Issues found:")
        for issue in results['issues']:
            print(f"    - {issue}")
    else:
        print(f"  No issues found")
    
    if 'label_distribution' in results['statistics']:
        print(f"  Label distribution: {results['statistics']['label_distribution']}")

## 9. Batch Processing and Collation

In [ ]:
def custom_collate_fn(batch: List[Dict]) -> Dict[str, torch.Tensor]:
    """
    Custom collation function for batching.
    
    Following batching strategies from:
        Micikevicius et al. (2018): "Mixed Precision Training"
    """
    # Handle text data
    if 'text' in batch[0]:
        texts = [item['text'] for item in batch]
        labels = torch.stack([item['labels'] for item in batch])
        
        return {
            'texts': texts,
            'labels': labels
        }
    
    # Handle tokenized data
    elif 'input_ids' in batch[0]:
        input_ids = torch.stack([item['input_ids'] for item in batch])
        attention_mask = torch.stack([item['attention_mask'] for item in batch])
        labels = torch.stack([item['labels'] for item in batch])
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }
    
    else:
        raise ValueError("Unknown batch format")

# Create DataLoader with custom collation
custom_loader = DataLoader(
    custom_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=custom_collate_fn
)

# Process a batch
batch = next(iter(custom_loader))

print("Custom Batch Processing:")
print(f"  Batch keys: {list(batch.keys())}")
print(f"  Batch size: {len(batch['labels'])}")
print(f"  Labels shape: {batch['labels'].shape}")
print(f"  Labels: {batch['labels'].tolist()}")

## 10. Performance Benchmarking

In [ ]:
def benchmark_dataloader(dataloader: DataLoader, num_batches: int = 100) -> Dict[str, float]:
    """
    Benchmark DataLoader performance.
    
    Following benchmarking methodology from:
        PyTorch Performance Tuning Guide
    """
    import time
    
    times = []
    
    # Warmup
    for _ in range(10):
        _ = next(iter(dataloader))
    
    # Benchmark
    for i, batch in enumerate(dataloader):
        if i >= num_batches:
            break
        
        start = time.perf_counter()
        # Simulate processing
        if torch.cuda.is_available():
            batch = {k: v.cuda(non_blocking=True) if torch.is_tensor(v) else v 
                    for k, v in batch.items()}
        end = time.perf_counter()
        
        times.append(end - start)
    
    return {
        'mean_time': np.mean(times),
        'std_time': np.std(times),
        'min_time': np.min(times),
        'max_time': np.max(times),
        'throughput': len(dataloader.dataset) / sum(times) if times else 0
    }

# Benchmark different configurations
configurations = [
    {'batch_size': 8, 'num_workers': 0},
    {'batch_size': 16, 'num_workers': 2},
    {'batch_size': 32, 'num_workers': 4}
]

print("DataLoader Performance Benchmarking:")
print("="*60)

for config in configurations:
    loader = DataLoader(
        custom_dataset,
        batch_size=config['batch_size'],
        num_workers=config['num_workers'],
        pin_memory=torch.cuda.is_available()
    )
    
    metrics = benchmark_dataloader(loader, num_batches=50)
    
    print(f"\nConfiguration: batch_size={config['batch_size']}, workers={config['num_workers']}")
    print(f"  Mean time per batch: {metrics['mean_time']*1000:.2f} ms")
    print(f"  Throughput: {metrics['throughput']:.0f} samples/sec")

## 11. Save and Load Processed Data

In [ ]:
# Save processed data
def save_processed_data(dataset: Dataset, filepath: Path):
    """
    Save processed dataset for future use.
    
    Following serialization best practices from:
        PyTorch Serialization Semantics Documentation
    """
    ensure_dir(filepath.parent)
    
    data_dict = {
        'texts': [],
        'labels': [],
        'metadata': {
            'num_classes': AG_NEWS_NUM_CLASSES,
            'class_names': AG_NEWS_CLASSES,
            'processed_date': pd.Timestamp.now().isoformat()
        }
    }
    
    # Extract data
    for i in range(len(dataset)):
        item = dataset[i]
        if 'text' in item:
            data_dict['texts'].append(item['text'])
        data_dict['labels'].append(
            item['labels'].item() if torch.is_tensor(item['labels']) else item['labels']
        )
    
    # Save
    safe_save(data_dict, filepath)
    print(f"Data saved to: {filepath}")
    
    return filepath

# Save sample data
from src.utils.io_utils import OUTPUT_DIR
output_path = OUTPUT_DIR / "tutorial" / "sample_data.json"
saved_path = save_processed_data(custom_dataset, output_path)

# Load saved data
loaded_data = safe_load(saved_path)
print(f"\nLoaded data verification:")
print(f"  Number of texts: {len(loaded_data['texts'])}")
print(f"  Number of labels: {len(loaded_data['labels'])}")
print(f"  Metadata: {loaded_data['metadata']}")

## 12. Conclusions and Next Steps

### Data Loading Summary

This tutorial demonstrated fundamental data loading concepts:

1. **Environment Setup**: Configured data loading environment with necessary libraries
2. **Multiple Sources**: Loaded data from processed files and Hugging Face Hub
3. **Custom Datasets**: Implemented PyTorch Dataset classes for flexibility
4. **DataLoader Configuration**: Created optimized data loaders with proper settings
5. **Memory Efficiency**: Implemented lazy loading for large datasets
6. **Data Validation**: Performed comprehensive validation checks
7. **Batch Processing**: Configured custom collation functions
8. **Performance Benchmarking**: Evaluated loading performance
9. **Data Persistence**: Saved and loaded processed data

### Key Takeaways

1. **Multiple Loading Methods**: Different sources require different approaches
2. **Memory Management**: Lazy loading essential for large datasets
3. **Validation Importance**: Always validate data before training
4. **Performance Optimization**: Proper DataLoader configuration significantly impacts speed
5. **Reproducibility**: Save processed data for consistent experiments

### Next Steps

1. **Advanced Data Loading**:
   - Implement streaming data loaders
   - Use distributed data loading
   - Apply data sharding strategies

2. **Optimization**:
   - Profile data loading bottlenecks
   - Implement prefetching strategies
   - Use memory mapping for large files

3. **Integration**:
   - Connect with preprocessing pipeline
   - Implement data augmentation
   - Create data versioning system

4. **Production**:
   - Build scalable data pipelines
   - Implement real-time data loading
   - Monitor data quality metrics

### References

For deeper understanding, consult:
- Data loading documentation: `docs/user_guide/data_preparation.md`
- Preprocessing: `notebooks/tutorials/02_preprocessing_tutorial.ipynb`
- Model training: `notebooks/tutorials/03_model_training_basics.ipynb`
- Advanced techniques: `notebooks/tutorials/05_prompt_engineering.ipynb`